# ChronoTick 2: TimesFM 2.5 Fine-Tuning

Fine-tune Google TimesFM 2.5 (200M) on univariate clock drift data.
FT is strictly univariate; sensor covariates are only used at inference
via XReg (ridge regression wrapper).

## Experiments
- **E1_uni_uni**: Univariate train + univariate inference (`lr=1e-4`)
- **E2_uni_xreg**: Same checkpoint as E1 + XReg at inference (no retraining)
- **E3_uni_xreg_lr5**: Lower LR training (`lr=5e-5`) + XReg inference

All share: `num_epochs=50`, `batch_size=64`, `context_length=128`.

E1 and E2 produce identical checkpoints; only evaluation differs.
The `train_key` field deduplicates training.

## Training Mode
Set `TRAINING_MODE` to "combined" or "per_machine".

In [ ]:
# === Environment Setup ===
import os
import subprocess
import sys

IN_COLAB = "COLAB_GPU" in os.environ or os.path.exists("/content")

if IN_COLAB:
    REPO_DIR = "/content/sensor-collector"
    REPO_URL = "https://github.com/JaimeCernuda/sensor-collector.git"
    GITHUB_TOKEN = None
    try:
        from google.colab import userdata

        GITHUB_TOKEN = userdata.get("GITHUB_TOKEN")
    except Exception:
        print("WARNING: GITHUB_TOKEN not available")
    auth_url = (
        f"https://{GITHUB_TOKEN}@github.com/JaimeCernuda/sensor-collector.git"
        if GITHUB_TOKEN
        else REPO_URL
    )
    if os.path.exists(REPO_DIR):
        subprocess.run(
            ["git", "-C", REPO_DIR, "remote", "set-url", "origin", auth_url], check=True
        )
        subprocess.run(["git", "-C", REPO_DIR, "fetch", "-q", "origin"], check=True)
        subprocess.run(
            ["git", "-C", REPO_DIR, "reset", "--hard", "origin/main"], check=True
        )
    else:
        subprocess.run(["git", "clone", "-q", auth_url, REPO_DIR], check=True)
    subprocess.run(
        ["git", "-C", REPO_DIR, "config", "user.name", "Colab Runner"], check=True
    )
    subprocess.run(
        ["git", "-C", REPO_DIR, "config", "user.email", "colab@chronotick.dev"],
        check=True,
    )
    subprocess.run(["pip", "install", "-q", "-e", f"{REPO_DIR}/tick2/"], check=True)
    tick2_src = f"{REPO_DIR}/tick2/src"
    if tick2_src not in sys.path:
        sys.path.insert(0, tick2_src)

    # Always mount Drive — needed for checkpoint persistence (models too large for git)
    from google.colab import drive

    drive.mount("/content/drive")

    # Data: prefer repo copy, fall back to Drive
    DATA_DIR = f"{REPO_DIR}/sensors/data"
    if not os.path.isdir(f"{DATA_DIR}/24h_snapshot"):
        DATA_DIR = "/content/drive/MyDrive/chronotick2/data"

    RESULTS_DIR = f"{REPO_DIR}/tick2/notebooks/output/03"
else:
    GITHUB_TOKEN = None
    DATA_DIR = None
    RESULTS_DIR = os.path.join(
        os.path.dirname("__file__") if "__file__" in dir() else ".", "output", "03"
    )

DEVICE_DIR_MAP = {"cuda": "gpu", "cpu": "cpu"}


def checkpoint_push(label):
    """Git add, commit, and push notebook 03d results."""
    if not IN_COLAB:
        return
    try:
        subprocess.run(
            ["git", "-C", REPO_DIR, "add", "tick2/notebooks/output/03/"],
            check=True,
            capture_output=True,
        )
        status = subprocess.run(
            [
                "git",
                "-C",
                REPO_DIR,
                "status",
                "--porcelain",
                "tick2/notebooks/output/03/",
            ],
            capture_output=True,
            text=True,
        )
        if not status.stdout.strip():
            return
        subprocess.run(
            [
                "git",
                "-C",
                REPO_DIR,
                "commit",
                "-m",
                f"results: notebook 03d timesfm {label} ({device_label})",
            ],
            check=True,
            capture_output=True,
        )
        if GITHUB_TOKEN:
            subprocess.run(
                ["git", "-C", REPO_DIR, "fetch", "-q", "origin"],
                capture_output=True,
                timeout=30,
            )
            subprocess.run(
                ["git", "-C", REPO_DIR, "rebase", "origin/main"],
                capture_output=True,
                timeout=30,
            )
            subprocess.run(
                ["git", "-C", REPO_DIR, "push"],
                check=True,
                capture_output=True,
                timeout=60,
            )
            print(f"  [CHECKPOINT] Pushed {label}")
    except Exception as e:
        print(f"  [CHECKPOINT WARNING] {e}")


print(f"Environment: {'Colab' if IN_COLAB else 'Local'}")

In [ ]:
# === Install TimesFM Dependencies ===
if IN_COLAB:
    subprocess.run(
        [
            "git",
            "clone",
            "-q",
            "--depth",
            "1",
            "https://github.com/google-research/timesfm",
            "/content/timesfm",
        ],
        capture_output=True,
    )
    subprocess.run(
        ["touch", "/content/timesfm/src/timesfm/timesfm_2p5/__init__.py"],
        check=True,
    )
    subprocess.run(
        ["pip", "install", "-q", "/content/timesfm[torch]"],
        check=True,
    )

import timesfm

assert hasattr(timesfm, "TimesFM_2p5_200M_torch"), "TimesFM 2.5 not available"
print("timesfm ready")

In [ ]:
# === Imports, Config & Training Mode ===
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import torch

from tick2.finetuning.base import FineTuneConfig
from tick2.finetuning.data_prep import prepare_datasets
from tick2.finetuning.evaluate import (
    compare_ft_vs_zero_shot,
    evaluate_finetuned,
    load_zero_shot_baselines,
)
from tick2.finetuning.timesfm_ft import finetune_timesfm, load_finetuned_timesfm
from tick2.utils.gpu import clear_gpu_memory

sns.set_theme(style="whitegrid", font_scale=1.1)

# --- User Configuration ---
TRAINING_MODE = "combined"  # "combined" or "per_machine"
DEVICE_OVERRIDE = None  # None = auto-detect, "cuda", or "cpu"
FORCE_RETRAIN = False  # Set True to retrain even if cached results exist

# --- Derived settings ---
device = DEVICE_OVERRIDE or ("cuda" if torch.cuda.is_available() else "cpu")
device_label = DEVICE_DIR_MAP.get(device, device)
config = FineTuneConfig(
    context_length=1024,
    prediction_length=96,
    max_covariates=30,
    seed=42,
)
print(f"Device: {device}, Mode: {TRAINING_MODE}")

In [ ]:
# === Load and Prepare Data ===
data_dir = Path(DATA_DIR) if DATA_DIR else None
prepared = prepare_datasets(config, data_dir=data_dir)
for name, p in prepared.items():
    print(
        f"  {name:16s}: train={len(p.split.train)}, "
        f"val={len(p.split.val)}, test={len(p.split.test)}"
    )

In [ ]:
# === Fine-Tune TimesFM 2.5 (E1, E2, E3) ===
from tick2.finetuning.base import FineTuneResult
from tick2.utils.colab import (
    load_checkpoint_from_drive,
    save_checkpoint_to_drive,
    setup_training_log,
)

output_base = Path(RESULTS_DIR)
ft_output_dir = output_base / "timesfm_ft" / TRAINING_MODE
device_results_dir = output_base / device_label
device_results_dir.mkdir(parents=True, exist_ok=True)

log_path = setup_training_log(ft_output_dir)
print(f"Training log: {log_path}")

# --- Experiment definitions ---
# E1: Univariate train + univariate inference
# E2: Same checkpoint as E1 + XReg at inference (no retraining)
# E3: Lower LR training + XReg inference
# train_key deduplicates training: E1 and E2 share the "default" key
EXPERIMENTS = [
    {
        "name": "E1_uni_uni",
        "train_key": "default",
        "with_xreg_eval": False,
        "learning_rate": 1e-4,
        "num_epochs": 50,
        "batch_size": 64,
        "context_length": 128,
        "horizon_length": 32,
    },
    {
        "name": "E2_uni_xreg",
        "train_key": "default",
        "with_xreg_eval": True,
        "learning_rate": 1e-4,
        "num_epochs": 50,
        "batch_size": 64,
        "context_length": 128,
        "horizon_length": 32,
    },
    {
        "name": "E3_uni_xreg_lr5",
        "train_key": "lr5",
        "with_xreg_eval": True,
        "learning_rate": 5e-5,
        "num_epochs": 50,
        "batch_size": 64,
        "context_length": 128,
        "horizon_length": 32,
    },
]

all_ft_results = []
experiment_labels = {}
trained_keys = {}  # train_key -> (ckpt_path, ft_results)

for exp in EXPERIMENTS:
    exp_name = exp["name"]
    train_key = exp["train_key"]
    print(f"\n{'=' * 60}")
    print(
        f"  {exp_name}  (train_key={train_key},"
        f" xreg_eval={exp['with_xreg_eval']},"
        f" lr={exp['learning_rate']})"
    )
    print(f"{'=' * 60}")

    exp_dir = ft_output_dir / exp_name
    cached_eval = device_results_dir / f"timesfm-2.5-ft-{exp_name}_{TRAINING_MODE}.csv"

    # If train_key already trained, reuse checkpoint
    # (E2 reuses E1's)
    if train_key in trained_keys:
        ckpt_path, prev_results = trained_keys[train_key]
        print(f"  [REUSE] Checkpoint from train_key={train_key}: {ckpt_path}")
        stub = FineTuneResult(
            model_name=f"timesfm-2.5-ft-{exp_name}",
            machine=TRAINING_MODE,
            checkpoint_path=str(ckpt_path),
            train_loss=(prev_results[0].train_loss if prev_results else []),
            val_loss=(prev_results[0].val_loss if prev_results else []),
            best_epoch=(prev_results[0].best_epoch if prev_results else 0),
            training_time_s=0.0,
            config=exp,
        )
        all_ft_results.append(stub)
        experiment_labels[exp_name] = [stub]
        continue

    # Check for cached eval or checkpoint
    ckpt_local = exp_dir / "combined" / "best_model.pt"
    ckpt_dir = exp_dir / "combined"

    if not ckpt_local.exists() and not FORCE_RETRAIN:
        drive_name = f"timesfm_ft/{TRAINING_MODE}/{exp_name}"
        resumed = load_checkpoint_from_drive(
            model_name=drive_name,
            local_path=str(ckpt_dir),
        )
        if resumed:
            print(f"  [RESUMED] From Drive: {resumed}")

    if cached_eval.exists() and not FORCE_RETRAIN:
        print(f"  [CACHED] Eval exists: {cached_eval}")
        stub = FineTuneResult(
            model_name=f"timesfm-2.5-ft-{exp_name}",
            machine=TRAINING_MODE,
            checkpoint_path=str(ckpt_dir),
            config=exp,
        )
        all_ft_results.append(stub)
        experiment_labels[exp_name] = [stub]
        trained_keys[train_key] = (ckpt_dir, [stub])
        continue

    if ckpt_local.exists() and not FORCE_RETRAIN:
        print(f"  [CACHED] Checkpoint: {ckpt_local}")
        stub = FineTuneResult(
            model_name=f"timesfm-2.5-ft-{exp_name}",
            machine=TRAINING_MODE,
            checkpoint_path=str(ckpt_dir),
            config=exp,
        )
        all_ft_results.append(stub)
        experiment_labels[exp_name] = [stub]
        trained_keys[train_key] = (ckpt_dir, [stub])
        continue

    clear_gpu_memory()

    try:
        ft_results = finetune_timesfm(
            prepared=prepared,
            config=config,
            output_dir=str(exp_dir),
            training_mode=TRAINING_MODE,
            learning_rate=exp["learning_rate"],
            num_epochs=exp["num_epochs"],
            batch_size=exp["batch_size"],
            context_length=exp["context_length"],
            horizon_length=exp["horizon_length"],
        )

        for r in ft_results:
            r.model_name = f"timesfm-2.5-ft-{exp_name}"
            print(f"  {r.machine}: {r.training_time_s:.1f}s")

        all_ft_results.extend(ft_results)
        experiment_labels[exp_name] = ft_results
        trained_keys[train_key] = (ckpt_dir, ft_results)

        save_checkpoint_to_drive(
            local_path=ckpt_dir,
            model_name=(f"timesfm_ft/{TRAINING_MODE}/{exp_name}"),
        )
        checkpoint_push(exp_name)

    except Exception as e:
        print(f"  [FAIL] {exp_name}: {e}")
        import traceback

        traceback.print_exc()
    finally:
        clear_gpu_memory()

print(f"\n{'=' * 60}")
print(f"  Completed: {list(experiment_labels.keys())}")
print(f"{'=' * 60}")

In [ ]:
# === Evaluate Fine-Tuned Models ===
from tick2.finetuning.data_prep import combine_training_data
from tick2.models.timesfm import TimesFMWrapper

# Compute shared feature intersection for XReg evaluation
_, shared_features_all = combine_training_data(prepared)
eval_features = shared_features_all[: config.max_covariates]
print(
    f"Shared eval features: {len(eval_features)}"
    f" (capped from {len(shared_features_all)})"
)

eval_dfs = []

for exp in EXPERIMENTS:
    exp_name = exp["name"]
    with_xreg = exp["with_xreg_eval"]
    print(f"\n--- Evaluating {exp_name} (xreg={with_xreg}) ---")

    cached_eval = device_results_dir / f"timesfm-2.5-ft-{exp_name}_{TRAINING_MODE}.csv"
    if cached_eval.exists() and not FORCE_RETRAIN:
        print(f"  [CACHED] {cached_eval}")
        eval_dfs.append(pd.read_csv(cached_eval))
        continue

    # Find checkpoint (may come from a shared train_key)
    exp_dir = ft_output_dir / exp_name
    ckpt_dir = exp_dir / "combined"
    if not ckpt_dir.exists():
        # Check if this experiment reuses another's checkpoint
        results_for_exp = experiment_labels.get(exp_name, [])
        if results_for_exp and results_for_exp[0].checkpoint_path:
            ckpt_dir = Path(results_for_exp[0].checkpoint_path)
    if not ckpt_dir.exists():
        print(f"  [SKIP] No checkpoint for {exp_name}")
        continue

    clear_gpu_memory()

    try:
        ft_model = load_finetuned_timesfm(str(ckpt_dir))

        ft_wrapper = TimesFMWrapper(
            model_name=f"timesfm-2.5-ft-{exp_name}",
        )
        ft_wrapper._model = ft_model
        ft_wrapper._device = device

        # E1 (univariate eval): no covariates
        # E2/E3 (XReg eval): pass shared features
        shared_cols = eval_features if with_xreg else None

        results_for_exp = experiment_labels.get(exp_name, [])
        ft_epochs = results_for_exp[0].best_epoch if results_for_exp else None
        ft_time = results_for_exp[0].training_time_s if results_for_exp else None
        ft_machines = results_for_exp[0].machine if results_for_exp else ""

        eval_df = evaluate_finetuned(
            model=ft_wrapper,
            prepared=prepared,
            config=config,
            training_mode=f"ft_{TRAINING_MODE}",
            ft_epochs=ft_epochs,
            ft_time_s=ft_time,
            ft_train_machines=ft_machines,
            shared_feature_cols=shared_cols,
        )

        if not eval_df.empty:
            eval_df["experiment"] = exp_name
            eval_df.to_csv(cached_eval, index=False)
            eval_dfs.append(eval_df)
            print(f"  MAE: {eval_df['mae'].mean():.4f}")
            print(f"  Saved: {cached_eval}")
        else:
            print(f"  [WARN] No eval results for {exp_name}")

        checkpoint_push(f"eval-{exp_name}")

    except Exception as e:
        print(f"  [FAIL] Eval {exp_name}: {e}")
        import traceback

        traceback.print_exc()
    finally:
        clear_gpu_memory()

# Combine all evaluation results
if eval_dfs:
    ft_eval_df = pd.concat(eval_dfs, ignore_index=True)
    print(f"\nTotal FT eval rows: {len(ft_eval_df)}")
    print(f"Mean MAE:  {ft_eval_df['mae'].mean():.4f}")
    print(f"Mean RMSE: {ft_eval_df['rmse'].mean():.4f}")
    if ft_eval_df["coverage"].notna().any():
        print(f"Mean Coverage: {ft_eval_df['coverage'].mean():.1%}")
    display(ft_eval_df)
else:
    ft_eval_df = pd.DataFrame()
    print("No evaluation results collected.")

In [ ]:
# === Load Zero-Shot Baselines ===
zs_dir = output_base.parent / "output" / "02"
zs_results = load_zero_shot_baselines(zs_dir, model_name="timesfm-2.5")
print(f"Zero-shot: {len(zs_results)} rows")

In [ ]:
# === Comparison: Fine-Tuned vs Zero-Shot ===
if not ft_eval_df.empty and not zs_results.empty:
    combined = compare_ft_vs_zero_shot(ft_eval_df, zs_results)

    # --- Per-experiment improvement vs best ZS ---
    best_zs = (
        zs_results.groupby("machine")["mae"]
        .agg(["min", "idxmin"])
        .rename(columns={"min": "best_zs_mae"})
    )
    best_zs["best_zs_ctx"] = zs_results.loc[best_zs["idxmin"], "context_length"].values
    best_zs = best_zs.drop(columns=["idxmin"])

    summary_rows = []
    for machine in ft_eval_df["machine"].unique():
        if machine not in best_zs.index:
            continue
        bzs_mae = best_zs.loc[machine, "best_zs_mae"]
        bzs_ctx = int(best_zs.loc[machine, "best_zs_ctx"])

        for exp in EXPERIMENTS:
            exp_name = exp["name"]
            with_xreg = exp["with_xreg_eval"]

            # Filter to matching covariate mode:
            # E1 (univariate) -> with_covariates=False rows
            # E2/E3 (XReg)    -> with_covariates=True rows
            ft_mask = (
                ft_eval_df["model"].str.contains(exp_name, na=False)
                & (ft_eval_df["machine"] == machine)
                & (ft_eval_df["with_covariates"] == with_xreg)
            )
            if not ft_mask.any():
                continue
            ft_mae = ft_eval_df.loc[ft_mask, "mae"].mean()

            if bzs_mae > 0:
                imp = (bzs_mae - ft_mae) / bzs_mae * 100
                summary_rows.append(
                    {
                        "machine": machine,
                        "experiment": exp_name,
                        "with_xreg": with_xreg,
                        "ft_mae": ft_mae,
                        "best_zs_ctx": bzs_ctx,
                        "best_zs_mae": bzs_mae,
                        "vs_best_zs_pct": imp,
                    }
                )

    if summary_rows:
        summary_df = pd.DataFrame(summary_rows)
        print("=== FT vs Best Zero-Shot ===")
        display(summary_df.round(4))

        print("\n=== Per-Experiment Summary ===")
        for exp in EXPERIMENTS:
            exp_name = exp["name"]
            ed = summary_df[summary_df["experiment"] == exp_name]
            if not ed.empty:
                ft_m = ed["ft_mae"].mean()
                zs_m = ed["best_zs_mae"].mean()
                imp_m = ed["vs_best_zs_pct"].mean()
                xreg_label = "XReg" if exp["with_xreg_eval"] else "uni"
                print(
                    f"  {exp_name} ({xreg_label}): FT={ft_m:.4f},"
                    f" ZS={zs_m:.4f},"
                    f" improvement={imp_m:+.1f}%"
                )
    else:
        print("Could not compute improvement.")

    print(f"\nCombined results: {len(combined)} rows")
    display(combined)
elif ft_eval_df.empty:
    combined = pd.DataFrame()
    print("No FT results to compare.")
else:
    combined = ft_eval_df.copy()
    print("No zero-shot baselines to compare against.")
    display(ft_eval_df)

In [ ]:
# === Visualizations ===
fig_dir = output_base / "figures"
fig_dir.mkdir(parents=True, exist_ok=True)

if not ft_eval_df.empty:
    # --- 1. MAE Comparison Bar Chart (multi-experiment) ---
    fig, ax = plt.subplots(figsize=(14, 5))

    plot_rows = []

    # Add ZS baseline
    if not zs_results.empty:
        for machine in zs_results["machine"].unique():
            m_zs = zs_results[zs_results["machine"] == machine]
            plot_rows.append(
                {
                    "machine": machine,
                    "variant": "Zero-Shot",
                    "mae": m_zs["mae"].mean(),
                }
            )

    # Add FT experiments (filter to representative covariate mode)
    for exp in EXPERIMENTS:
        exp_name = exp["name"]
        with_xreg = exp["with_xreg_eval"]
        exp_data = ft_eval_df[
            ft_eval_df["model"].str.contains(exp_name, na=False)
            & (ft_eval_df["with_covariates"] == with_xreg)
        ]
        for machine in exp_data["machine"].unique():
            m_ft = exp_data[exp_data["machine"] == machine]
            plot_rows.append(
                {
                    "machine": machine,
                    "variant": exp_name,
                    "mae": m_ft["mae"].mean(),
                }
            )

    if plot_rows:
        plot_df = pd.DataFrame(plot_rows)
        sns.barplot(
            data=plot_df,
            x="machine",
            y="mae",
            hue="variant",
            ax=ax,
        )
        ax.set_ylabel("MAE (ppm)")
        ax.set_title("TimesFM 2.5: FT vs Zero-Shot MAE by Machine")
        ax.legend(
            title="Variant",
            bbox_to_anchor=(1.05, 1),
            loc="upper left",
        )
        plt.tight_layout()
        fig.savefig(
            fig_dir / "timesfm_ft_vs_zs_mae.png",
            dpi=150,
            bbox_inches="tight",
        )
        plt.show()
    else:
        plt.close(fig)

    # --- 2. Coverage Comparison (if available) ---
    if ft_eval_df["coverage"].notna().any():
        cov_data = ft_eval_df[ft_eval_df["coverage"].notna()]
        fig, ax = plt.subplots(figsize=(10, 5))
        sns.barplot(
            data=cov_data,
            x="machine",
            y="coverage",
            hue="model",
            ax=ax,
        )
        ax.axhline(
            0.8,
            color="red",
            linestyle="--",
            alpha=0.5,
            label="80% target",
        )
        ax.set_ylabel("Coverage")
        ax.set_title("Prediction Interval Coverage")
        ax.legend(
            title="Model",
            bbox_to_anchor=(1.05, 1),
            loc="upper left",
        )
        plt.tight_layout()
        fig.savefig(
            fig_dir / "timesfm_ft_coverage.png",
            dpi=150,
            bbox_inches="tight",
        )
        plt.show()

    print(f"Saved figures to: {fig_dir}")
else:
    print("No results to visualize.")

In [ ]:
# === Export Results ===
from tick2.benchmark.reporting import results_to_latex, save_results

if not ft_eval_df.empty:
    # Save combined FT results
    ft_csv = device_results_dir / f"timesfm-2.5-ft-all_{TRAINING_MODE}.csv"
    ft_eval_df.to_csv(ft_csv, index=False)
    print(f"FT results CSV: {ft_csv}")

if not combined.empty:
    csv_path, latex_path = save_results(
        combined,
        output_base,
        prefix=f"timesfm_ft_{TRAINING_MODE}",
    )
    print(f"Comparison CSV:   {csv_path}")
    print(f"Comparison LaTeX: {latex_path}")
    latex = results_to_latex(
        combined,
        caption=f"TimesFM 2.5 fine-tuning vs zero-shot ({TRAINING_MODE})",
        label="tab:timesfm-ft",
    )
    print(f"\n{latex}")
else:
    print("No results to export.")

In [ ]:
# === Final Push ===
if IN_COLAB:
    os.chdir(REPO_DIR)

    subprocess.run(
        ["git", "add", "tick2/notebooks/output/03/"],
        check=True,
    )

    status = subprocess.run(
        ["git", "status", "--porcelain", "tick2/notebooks/output/03/"],
        capture_output=True,
        text=True,
    )
    if status.stdout.strip():
        msg = f"results: notebook 03d timesfm-ft figures and combined ({device_label})"
        subprocess.run(
            ["git", "commit", "-m", msg],
            check=True,
        )
        if GITHUB_TOKEN:
            subprocess.run(
                ["git", "fetch", "-q", "origin"],
                capture_output=True,
                timeout=30,
            )
            subprocess.run(
                ["git", "rebase", "origin/main"],
                capture_output=True,
                timeout=30,
            )
            subprocess.run(["git", "push"], check=True)
            print("Pushed final outputs to GitHub.")
        else:
            print("Committed locally (no token for push).")
    else:
        print("No new outputs to commit.")
else:
    print(f"Local run. Outputs saved to: {output_base}")
    print(
        "Run 'git add tick2/notebooks/output/03/ && git commit && git push' to share."
    )